Table 5.2.3: Total individuals in settings (overlap between 5.2.1 and 5.2.2)

In [2]:
from gssutils import *

if is_interactive():
    import requests
    from cachecontrol import CacheControl
    from cachecontrol.caches.file_cache import FileCache
    from cachecontrol.heuristics import LastModified
    from pathlib import Path

    session = CacheControl(requests.Session(),
                           cache=FileCache('.cache'),
                           heuristic=LastModified())

    sourceFolder = Path('in')
    sourceFolder.mkdir(exist_ok=True)

    inputURL = 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/752515/AdultSubstanceMisuseNDTMSDataTables2017-18.xlsx'
    inputFile = sourceFolder / 'AdultSubstanceMisuseNDTMSDataTables2017-18.xlsx'
    response = session.get(inputURL)
    with open(inputFile, 'wb') as f:
      f.write(response.content)    

In [3]:
tab = loadxlstabs(inputFile, sheetids='Table 5.2.3')[0]

Loading in\AdultSubstanceMisuseNDTMSDataTables2017-18.xlsx which has size 272149 bytes
Table names: ['Table 5.2.3']


https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/752515/AdultSubstanceMisuseNDTMSDataTables2017-18.xlsx

In [4]:
observations = tab.excel_ref('B5').expand(DOWN).expand(RIGHT).is_not_blank()
observations

{<B6 5980.0>, <B5 12193.0>}

In [5]:
Setting = tab.excel_ref('A5').expand(DOWN).is_not_blank()
Setting

{<A5 'Inpatient unit'>, <A6 'Residential'>}

In [6]:
Dimensions = [
            HDimConst('Substance','All'),
            HDim(Setting,'Setting',DIRECTLY,LEFT),
            HDimConst('Clients in Treatment','All'),
            HDimConst('MeasureType','Count'),
            HDimConst('Unit','People')            
            ]

In [7]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
# if is_interactive():
#     savepreviewhtml(c1)

In [8]:
new_table = c1.topandas()
new_table

,OBS,Substance,Setting,Clients in Treatment,MeasureType,Unit
0,12193.0,All,Inpatient unit,All,Count,People
1,5980.0,All,Residential,All,Count,People


In [9]:
new_table.columns = ['Value' if x=='OBS' else x for x in new_table.columns]

In [10]:
new_table.head()

,Value,Substance,Setting,Clients in Treatment,MeasureType,Unit
0,12193.0,All,Inpatient unit,All,Count,People
1,5980.0,All,Residential,All,Count,People


In [11]:
new_table.dtypes

Value                   float64
Substance                object
Setting                  object
Clients in Treatment     object
MeasureType              object
Unit                     object
dtype: object

In [12]:
new_table['Value'] = new_table['Value'].astype(str)

In [13]:
new_table.head(3)

,Value,Substance,Setting,Clients in Treatment,MeasureType,Unit
0,12193.0,All,Inpatient unit,All,Count,People
1,5980.0,All,Residential,All,Count,People


In [25]:
new_table = new_table[['Substance','Setting','Clients in Treatment','Measure Type','Value','Unit']]

In [26]:
if is_interactive():
    SubstancetinationFolder = Path('out')
    SubstancetinationFolder.mkdir(exist_ok=True, parents=True)
    new_table.to_csv(SubstancetinationFolder / ('table5.2.3.csv'), index = False)